# CSS Knowledge

In [7]:
!pip install scrapy
import scrapy
from scrapy import Selector
import requests

url = "https://books.toscrape.com/"
response = requests.get(url)
sel = Selector(text=response.text)
books = sel.css("article.product_pod")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.7/331.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.3 MB/s eta 0:00:00


In [8]:
books

[<Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' '), ' product_pod ')]" data='<article class="product_pod">\n       ...'>,
 <Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' '), ' product_pod ')]" data='<article class="product_pod">\n       ...'>,
 <Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' '), ' product_pod ')]" data='<article class="product_pod">\n       ...'>,
 <Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' '), ' product_pod ')]" data='<article class="product_pod">\n       ...'>,
 <Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' '), ' product_pod ')]" data='<article class="product_pod">\n       ...'>,
 <Selector query="descendant-or-self::article[@class and contains(concat(' ', normalize-space(@class), ' 

In [9]:
def scrape_books_simple():
    #Ambil halaman dengan requests
    url = "https://books.toscrape.com/"
    response = requests.get(url)
    # Parse dengan Scrapy Selector
    selector = Selector(text=response.text)
    # Extract data buku dengan CSS selector
    books = selector.css("article.product_pod")
    results = []
    for book in books:
        data = {
            "judul": book.css("h3 a::attr(title)").get(),
            "harga": book.css("div.product_price p.price_color::text").get(),
            "rating": book.css("p.star-rating::attr(class)").get().split()[1]
        }
        results.append(data)
    return results

# Jalankan scraping dan tampilkan hasil
book_results = scrape_books_simple()

In [10]:
book_results

[{'judul': 'A Light in the Attic', 'harga': 'Â£51.77', 'rating': 'Three'},
 {'judul': 'Tipping the Velvet', 'harga': 'Â£53.74', 'rating': 'One'},
 {'judul': 'Soumission', 'harga': 'Â£50.10', 'rating': 'One'},
 {'judul': 'Sharp Objects', 'harga': 'Â£47.82', 'rating': 'Four'},
 {'judul': 'Sapiens: A Brief History of Humankind',
  'harga': 'Â£54.23',
  'rating': 'Five'},
 {'judul': 'The Requiem Red', 'harga': 'Â£22.65', 'rating': 'One'},
 {'judul': 'The Dirty Little Secrets of Getting Your Dream Job',
  'harga': 'Â£33.34',
  'rating': 'Four'},
 {'judul': 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  'harga': 'Â£17.93',
  'rating': 'Three'},
 {'judul': 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  'harga': 'Â£22.60',
  'rating': 'Four'},
 {'judul': 'The Black Maria', 'harga': 'Â£52.15', 'rating': 'One'},
 {'judul': 'Starving Hearts (Triangular Trade Trilogy, #1)',
  'harga': 'Â£13.99',
  'r

In [11]:
import pandas as pd
df = pd.DataFrame(book_results)
df

,judul,harga,rating
0,A Light in the Attic,Â£51.77,Three
1,Tipping the Velvet,Â£53.74,One
2,Soumission,Â£50.10,One
3,Sharp Objects,Â£47.82,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,Five
5,The Requiem Red,Â£22.65,One
6,The Dirty Little Secrets of Getting Your Dream...,Â£33.34,Four
7,The Coming Woman: A Novel Based on the Life of...,Â£17.93,Three
8,The Boys in the Boat: Nine Americans and Their...,Â£22.60,Four
9,The Black Maria,Â£52.15,One


# Use Case dengan Spider